In [1]:
!pip3 install gdown
!python3 -m spacy download en
!gdown 1u2pzjXIuhqf6CR2CBI6kqxLtiynLhUEE #gender_name_csv_file
!pip3 install spacytextblob langdetect

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
     |################################| 13.9 MB 3.4 MB/s eta 0:00:01
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Downloading...
From: https://drive.google.com/uc?id=1u2pzjXIuhqf6CR2CBI6kqxLtiynLhUEE
To: /home/vkumar364/goodreads_id_name_gender_new.csv
100%|#######################################| 98.5M/98.5M [00:00<00:00, 127MB/s]


In [2]:
!pip3 install langid

In [1]:
import gzip
import json
import os
import pandas as pd
import langid

def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)        
    return data

In [2]:
!gdown 1M5iqCZ8a7rZRtsmY5KQ5rYnP9S0bQJVo

Downloading...
From: https://drive.google.com/uc?id=1M5iqCZ8a7rZRtsmY5KQ5rYnP9S0bQJVo
To: /home/vkumar364/goodreads_reviews_young_adult.json.gz
100%|#########################################| 899M/899M [00:07<00:00, 114MB/s]


In [8]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           187G        110G        3.1G        289M         74G         75G
Swap:            0B          0B          0B


In [9]:
def download_and_join_data(ggid,csvname,gendered_df):
#     os.system('gdown '+ ggid)
    data = load_data(csvname)
    df = pd.DataFrame(data)
    final = pd.merge(df, gendered_data, on='user_id', how='left')
    return final[['user_id', 'rating', 'review_text', 'n_votes','n_comments','gender']]

In [10]:
gendered_data = pd.read_csv('goodreads_id_name_gender_new.csv')
gendered_data = gendered_data.drop(columns=['Unnamed: 0','name_x','age','location','friends_count','groups_count'])
gendered_data = gendered_data.rename(columns={'uuid':'user_id'})
gendered_data.head()

,id,username,joined,reviews_count,first_name,user_id,gender
0,50002191,sozerseda,12/2015,184.0,seda,aad6531213b6711ec6bfa5cd3ecac285,female
1,58298653,NaN,08/2016,24.0,olivia,30a37d704d9c28681606a3fbe80ed58f,female
2,36210433,NaN,10/2014,379.0,naomi,f856f6ef2eb4ad702af863175f281f92,female
3,6949799,azima,11/2011,230.0,azima,32d2876b7eb3db5a1ee87f29c01dce4c,female
4,21443818,NaN,NaN,NaN,nicholas,2ace7a7c6edcdd8ac788fe4fd552b7ca,male


In [11]:
selected = download_and_join_data('1THnnmE4XSCvMkGOsqapQr2cJI5amKA6X','goodreads_reviews_young_adult.json.gz', gendered_data)

In [12]:
selected.head()

,user_id,rating,review_text,n_votes,n_comments,gender
0,8842281e1d1347389f2ab93d60773d4d,5,I cracked and finally picked this up. Very enj...,24,25,male
1,7504b2aee1ecb5b2872d3da381c6c91e,5,I read this book because my fifth grade son wa...,0,0,female
2,f8a89075dc6de14857561522e729f82c,4,Though the book started out slow and only star...,0,0,female
3,f8a89075dc6de14857561522e729f82c,2,"*Update - 10/27/13* - After some sleep, I thin...",0,0,female
4,f8a89075dc6de14857561522e729f82c,5,"This is a moving, heartbreaking, view into a l...",0,0,female


In [13]:
selected_final = selected[selected['gender']!='UNK']

In [14]:
selected_final['gender'].value_counts()

female    1750500
male       261650
Name: gender, dtype: int64

In [15]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
!pip install contractions

[nltk_data] Downloading package stopwords to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [17]:
import contractions 
import re
from nltk.corpus import stopwords
import numpy as np

In [18]:
expand_contractions = True
remove_stopwords = False
format_text = True
tokenize = False
lemmatize = False

def clean_text(text):
    '''Text Preprocessing '''
    
    # Convert words to lower case
    text = text.lower()
    split_text = text.split()
    
    # Expand contractions
    if expand_contractions:
        expanded_words = []   
        for word in split_text:
            expanded_words.append(contractions.fix(word))
        text = " ".join(expanded_words)

    # Format words and remove unwanted characters
    if format_text:
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)
    
    # remove stopwords
    if remove_stopwords:
        split_text = text.split()
        stops = set(stopwords.words("english"))
        word = [w for w in split_text if not w in stops]
        text = " ".join(word)

    # Tokenize each word
    if tokenize:
        text =  nltk.WordPunctTokenizer().tokenize(text)
    
    # Lemmatize each token
    if lemmatize:
        lemm = nltk.stem.WordNetLemmatizer()
        text = list(map(lambda word:list(map(lemm.lemmatize, word)), text))
    
    return text

In [19]:
selected['review_length'] = selected['review_text'].apply(lambda x: len(x.split(' ')))

In [37]:
def get_subsampled_gendered(selected_final,lower,upper):
    selected_final_male = selected_final[selected_final['gender']=='male']
    selected_final_female = selected_final[selected_final['gender']=='female']
    selected_final_male_rev = selected_final_male[selected_final_male['review_length']>lower]
    selected_final_male_rev = selected_final_male_rev[selected_final_male_rev['review_length']<=upper]
    selected_final_female_rev = selected_final_female[selected_final_female['review_length']>lower]
    selected_final_female_rev = selected_final_female_rev[selected_final_female_rev['review_length']<=upper]
    #subsampling here
    min_select = min(list(selected_final_female_rev['rating'].value_counts()) + list(selected_final_male_rev['rating'].value_counts()))
    print(min_select)
    #sampling down the total data - for males
    x = selected_final_male_rev[selected_final_male_rev['rating']==1].sample(min_select)
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==2].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==3].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==4].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==5].sample(min_select))
    #sampling down the total data - for females
    y = selected_final_female_rev[selected_final_female_rev['rating']==1].sample(min_select)
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==2].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==3].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==4].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==5].sample(min_select))
    return x,y
    

In [38]:
male_rev,female_rev = get_subsampled_gendered(selected,5,500)

6795


In [22]:
def clean_en_sentiment(df):
    df['review_text_cleaned'] = df['review_text'].apply(clean_text)
    df['review_lang'] = df['review_text'].apply(lambda x: langid.classify(x)[0])
    df = df[df['review_lang'] == 'en']
    df['sentiment_score'] = df['review_text'].apply(lambda x: nlp(x)._.polarity)
    return df 

In [23]:
male_rev_cleaned = clean_en_sentiment(male_rev)
female_rev_cleaned = clean_en_sentiment(female_rev)

/tmp/ipykernel_30990/1754966298.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_score'] = df['review_text'].apply(lambda x: nlp(x)._.polarity)


In [24]:
female_rev_cleaned = female_rev_cleaned.reset_index()
male_rev_cleaned = male_rev_cleaned.reset_index()

In [25]:
male_rev_cleaned = male_rev_cleaned[['rating','n_votes', 'n_comments', 'gender','review_length','sentiment_score']]
female_rev_cleaned = female_rev_cleaned[['rating','n_votes', 'n_comments', 'gender','review_length','sentiment_score']]

In [26]:
male_rev_cleaned

,rating,n_votes,n_comments,gender,review_length,sentiment_score
0,1,0,0,male,27,-0.338000
1,1,0,0,male,391,0.122650
2,1,0,0,male,105,-0.143282
3,1,0,0,male,173,-0.121569
4,1,8,0,male,34,0.237500
...,...,...,...,...,...,...
31172,5,1,0,male,467,0.191668
31173,5,0,0,male,59,0.129464
31174,5,0,1,male,12,0.550000
31175,5,0,0,male,72,0.360633


In [27]:
male_rev_cleaned['gender'] = 0
female_rev_cleaned['gender'] = 1

In [28]:
combined_data = male_rev_cleaned.append(female_rev_cleaned)

In [29]:
combined_data

,rating,n_votes,n_comments,gender,review_length,sentiment_score
0,1,0,0,0,27,-0.338000
1,1,0,0,0,391,0.122650
2,1,0,0,0,105,-0.143282
3,1,0,0,0,173,-0.121569
4,1,8,0,0,34,0.237500
...,...,...,...,...,...,...
32066,5,1,0,1,432,0.261804
32067,5,0,0,1,11,0.527500
32068,5,0,0,1,47,-0.005000
32069,5,0,0,1,252,0.271875


In [30]:
final_g = combined_data[['n_votes', 'n_comments', 'gender', 'review_length','sentiment_score']]
final_w = combined_data[['n_votes', 'n_comments', 'review_length','sentiment_score']]
pred = combined_data['rating']

In [31]:
final_g.head()

,n_votes,n_comments,gender,review_length,sentiment_score
0,0,0,0,27,-0.338000
1,0,0,0,391,0.122650
2,0,0,0,105,-0.143282
3,0,0,0,173,-0.121569
4,8,0,0,34,0.237500


In [32]:
import statsmodels.api as sm
model_g = sm.OLS(pred, sm.add_constant(final_g)).fit()
model_w = sm.OLS(pred, sm.add_constant(final_w)).fit()

/nethome/vkumar364/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [33]:
model_g.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     3306.
Date:                Thu, 28 Apr 2022   Prob (F-statistic):               0.00
Time:                        16:48:34   Log-Likelihood:            -1.0448e+05
No. Observations:               63248   AIC:                         2.090e+05
Df Residuals:                   63242   BIC:                         2.090e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.5919      0.009    275.144      0.000       2.573       2.610
n_votes             0.0016      0.001      2.329      0.020       0.000       0.003
n_comments         -0.0200      0.002     -8.207      0.000      -0.025      -0.015
gender             -0.0148      0.010     -1.474      0.141      -0.034       0.005
review_length       0.0001   4.33e-05      3.305      0.001    5.83e-05       0.000
sentiment_score     2.7581      0.022    127.900      0.000       2.716       2.800
==============================================================================
Omnibus:                     4048.735   Durbin-Watson:                   0.412
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1469.379
Skew:                           0.055   Prob(JB):                         0.00
Kurtosis:                       2.261   Cond. No.                         718.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
sm.stats.anova_lm(model_w, model_g)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,63243.0,100805.417242,0.0,NaN,NaN,NaN
1,63242.0,100801.955552,1.0,3.46169,2.171825,0.140564


In [36]:
import gc
gc.collect()

96